# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
# Your code here; create the connection
con = sqlite3.Connection("C:/Users/14153/Desktop/data2.sqlite")

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
# Your code here
q0 = """
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN (
SELECT customerNumber 
FROM orders
WHERE orderDate = '2003-01-31');
"""
pd.read_sql(q0, con)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
q1 = """
SELECT products.productName, COUNT(orderdetails.orderNumber) AS 'Total_number_of_orders'
FROM products
JOIN orderdetails
ON products.productCode = orderdetails.productCode
GROUP BY productName
ORDER BY Total_number_of_orders;
"""
pd.read_sql(q1, con)

,productName,Total_number_of_orders
0,1952 Citroen-15CV,24
1,1957 Ford Thunderbird,24
2,1911 Ford Town Car,25
3,1917 Grand Touring Sedan,25
4,1932 Alfa Romeo 8C2300 Spider Sport,25
...,...,...
104,Diamond T620 Semi-Skirted Tanker,28
105,F/A 18 Hornet 1/72,28
106,HMS Bounty,28
107,P-51-D Mustang,28


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [5]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q2 = """
SELECT DISTINCT COUNT(orders.customerNumber) AS 'Total_number_of_people', products.productName
FROM orders
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
JOIN products ON orderdetails.productCode = products.productCode
GROUP BY productName
ORDER BY Total_number_of_people DESC;
"""

pd.read_sql(q2, con)

,Total_number_of_people,productName
0,53,1992 Ferrari 360 Spider red
1,28,P-51-D Mustang
2,28,HMS Bounty
3,28,F/A 18 Hornet 1/72
4,28,Diamond T620 Semi-Skirted Tanker
...,...,...
104,25,1932 Alfa Romeo 8C2300 Spider Sport
105,25,1917 Grand Touring Sedan
106,25,1911 Ford Town Car
107,24,1957 Ford Thunderbird


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [6]:
# Your code here
q3 = """
SELECT employees.employeeNumber, employees.firstName, employees.lastName, offices.city, employees.officeCode
FROM employees
JOIN offices ON employees.officeCode = offices.officeCode
WHERE employees.employeeNumber IN (
  SELECT DISTINCT customers.salesRepEmployeeNumber
  FROM customers
  JOIN orders ON customers.customerNumber = orders.customerNumber
  JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
  GROUP BY customers.salesRepEmployeeNumber, orderdetails.productCode
  HAVING COUNT(DISTINCT orders.orderNumber) < 20
);"""

pd.read_sql(q3, con)


,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [7]:
# Your code here
q4 = """
SELECT employees.employeeNumber, employees.firstName, employees.lastName, COUNT(customers.customerNumber) AS Number_of_customers
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
HAVING AVG(customers.creditLimit) > 15000;"""

pd.read_sql(q4, con)

,employeeNumber,firstName,lastName,Number_of_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!